##Network that takes two values, performs arthmetic (addition,subtraction,multiplication, division). The Network has to decide which operator best describes the dataset.

Uses: Consider add,subtract,multiply,divide as multiple networks.

In [407]:
import jax 
import jax.numpy as jnp
import numpy as onp
from jax import grad, jit, vmap
import timeit

In [408]:
#Consider a set of operators
def add (input): 
  return input[0]+input[1]
def subtract(input): 
  return input[0]-input[1]
def multiply (input): 
  return input[0]*input[1]
def divide (input): 
  return input[0]/input[1]

In [409]:
#Q will have to be 4x4 (or the four types of calculations)
def forward(input,Q):
  temp=[]
  temp.append(multiply(input))
  temp.append(divide(input))
  temp.append(add(input))
  temp.append(subtract(input)) 
  temp=jnp.array(temp) 
 
  att = jax.nn.softmax(jnp.dot(temp,Q.T))
  expect = att[0]*temp[0] + att[1]*temp[1] + att[2]*temp[2] + att[3]*temp[3] #deciding between math functions
  return expect,att

def MSE(input,target,params): 
  pred,_ = forward(input,params)
  calc=0.5*(pred-target)**2
  return calc

gradient =grad(MSE,argnums=2)

In [369]:
def update_attention(Q,dQ,lr=0.1): 
  Q= Q-(lr*dQ)
  return Q

Dataset - choose your operator 

In [393]:
dataset = [[20.0,5.0],[2.0,62.0],[47.0,4.0],[19.0,4.0]] #you can change these.
dataset=jnp.array(dataset)
target=[]
for i in range(4):
  target.append(dataset[i][0]/dataset[i][1]) #Change operator to subtract,add,divide,multiply
target=jnp.array(target)

Training Loop

Parameters is Q, 4x4 matrix

In [404]:
onp.random.seed(1003)
Q_param=onp.random.randn(4,4)/1000

In [405]:
start_time = timeit.default_timer()
for i in range(15):
  for j in range(4): 
    dparams=gradient(dataset[j],target[j],Q_param) 
    Q_param= update_attention(Q_param,dparams,lr=0.000001)
  test=(MSE(dataset[1],target[1],Q_param)) 
  print (f'Epoch: ',i)
  print(f'MSE',test)
elapsed = timeit.default_timer() - start_time
print (f'elapsed time: ', elapsed, 's')  

Epoch:  0
MSE 3.7429316
Epoch:  1
MSE 0.5824855
Epoch:  2
MSE 0.23713034
Epoch:  3
MSE 0.121499464
Epoch:  4
MSE 0.07048179
Epoch:  5
MSE 0.044307042
Epoch:  6
MSE 0.029486086
Epoch:  7
MSE 0.02048031
Epoch:  8
MSE 0.014707546
Epoch:  9
MSE 0.010848264
Epoch:  10
MSE 0.008179056
Epoch:  11
MSE 0.0062803295
Epoch:  12
MSE 0.0048973444
Epoch:  13
MSE 0.0038695482
Epoch:  14
MSE 0.0030923556
elapsed time:  3.3275311509996754 s


Test, network has chosen the operator that describes dataset

In [406]:
pred,_=forward([50,25],Q_param)
print(pred)

2.0
